In [1]:
# def get_exchange_rate(amount, from_currency, to_currency):

In [7]:
import openai
import json
import os
from rich import print as rprint

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [17]:
amount = 100
from_currency = "USB"
to_currency = "KRW"

In [18]:
exchange_rates = {
    "USB_to_KRW": 1300,
    "EUR_to_KRW": 1400,
    }
exchange_rates

{'USB_to_KRW': 1300, 'EUR_to_KRW': 1400}

In [19]:
conversion_key = f"{from_currency}_to_{to_currency}"
conversion_key

'USB_to_KRW'

In [22]:
rate = exchange_rates.get(conversion_key)
rate

1300

In [23]:
converted_amount = amount * rate
converted_amount

130000

In [24]:
json.dumps({"amount": converted_amount, "currency": to_currency})

'{"amount": 130000, "currency": "KRW"}'

In [25]:
user_question = "100 USD를 KRW로 변환하면 얼마인가요?"

In [26]:
print(f"😎 사용자 질문: {user_question}\n")

😎 사용자 질문: 100 USD를 KRW로 변환하면 얼마인가요?



In [27]:
print("🧠 [1차 호출] AI, 이 질문을 보고 필요한 정보를 뽑아서 도구를 쓸지 결정해줘!")

🧠 [1차 호출] AI, 이 질문을 보고 필요한 정보를 뽑아서 도구를 쓸지 결정해줘!


In [29]:
response =client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": user_question,
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "한 통화의 금액을 다른 통화로 환전해줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {
                            "type": "number",
                            "description": "환전할 금액",
                        },
                        "from_currency": {
                            "type": "string",
                            "description": "환전의 기준이 되는 통화 (예: USD, EUR)",
                        },
                        "to_currency": {
                            "type": "string",
                            "description": "환전 후의 목표 통화 (예: KRW)",
                        }
                    },
                    "required": ["amount", "from_currency", "to_currency"],
                }
            }
        }
    ],
    tool_choice="auto",
)

In [31]:
rprint(response)

ChatCompletion(
    id='chatcmpl-BpEL0a8FEMbBg4MHIPqwqw3HaNjbL',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_edMctoOqGVsNfFeQeb2onuDF',
                        function=Function(
                            arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}',
                            name='get_exchange_rate'
                        ),
                        type='function'
                    )
                ],
                annotations=[]
            )
        )
    ],
    created=1751549542,
    model='gpt-4o-2024-08-06',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_07871e2ad8',
    usage=CompletionUsage(
        completion_tokens=26,
        prompt_tokens=114,
        total_tokens=140,
        prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0},
        completion_tokens_details={
            'reasoning_tokens': 0,
            'audio_tokens': 0,
            'accepted_prediction_tokens': 0,
            'rejected_prediction_tokens': 0
        }
    )
)

In [33]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_edMctoOqGVsNfFeQeb2onuDF', function=Function(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', name='get_exchange_rate'), type='function')], annotations=[])

In [36]:
print(response_message.tool_calls)

[ChatCompletionMessageToolCall(id='call_edMctoOqGVsNfFeQeb2onuDF', function=Function(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', name='get_exchange_rate'), type='function')]


In [39]:
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_edMctoOqGVsNfFeQeb2onuDF', function=Function(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', name='get_exchange_rate'), type='function')]

In [37]:
print("✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요!")

✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요!


In [40]:
tool_call = tool_calls[0]
tool_call

ChatCompletionMessageToolCall(id='call_edMctoOqGVsNfFeQeb2onuDF', function=Function(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', name='get_exchange_rate'), type='function')

In [42]:
function_args = json.loads(tool_call.function.arguments)
function_args

{'amount': 100, 'from_currency': 'USD', 'to_currency': 'KRW'}

In [43]:
print(f"   - AI가 추출한 정보: {function_args}\n")

   - AI가 추출한 정보: {'amount': 100, 'from_currency': 'USD', 'to_currency': 'KRW'}



In [44]:
messages = [
    {
        "role": "user",
        "content": user_question,
    },
    response_message,
]

In [45]:
messages

[{'role': 'user', 'content': '100 USD를 KRW로 변환하면 얼마인가요?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_edMctoOqGVsNfFeQeb2onuDF', function=Function(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}', name='get_exchange_rate'), type='function')], annotations=[])]

In [47]:
amount = function_args.get("amount")
from_currency = function_args.get("from_currency")
to_currency = function_args.get("to_currency")

100

In [51]:
function_response = json.dumps({"amount": converted_amount, "currency": to_currency})
function_response

'{"amount": 130000, "currency": "KRW"}'

In [52]:
print(f"📄 [도구 결과] 계산 결과: {function_response}\n")

📄 [도구 결과] 계산 결과: {"amount": 130000, "currency": "KRW"}



In [53]:
print("🧠 [2차 호출] AI, 계산 결과를 바탕으로 최종 답변을 만들어줘!")

🧠 [2차 호출] AI, 계산 결과를 바탕으로 최종 답변을 만들어줘!


In [56]:
tool_call.id

'call_edMctoOqGVsNfFeQeb2onuDF'

In [57]:
messages.append(
    {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": "get_exchange_rate",
        "content": function_response,
    }
)

In [59]:
rprint(messages)

[
    {'role': 'user', 'content': '100 USD를 KRW로 변환하면 얼마인가요?'},
    ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_edMctoOqGVsNfFeQeb2onuDF',
                function=Function(
                    arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW"}',
                    name='get_exchange_rate'
                ),
                type='function'
            )
        ],
        annotations=[]
    ),
    {
        'tool_call_id': 'call_edMctoOqGVsNfFeQeb2onuDF',
        'role': 'tool',
        'name': 'get_exchange_rate',
        'content': '{"amount": 130000, "currency": "KRW"}'
    }
]

In [61]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

In [63]:
rprint(second_response)

ChatCompletion(
    id='chatcmpl-BpEXoQAw92fqJqEVaFkOHDqMA3abN',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='100 USD는 약 130,000 KRW입니다. 환율은 변동될 수 있으니 확인이 필요합니다.',
                refusal=None,
                role='assistant',
                function_call=None,
                tool_calls=None,
                annotations=[]
            )
        )
    ],
    created=1751550336,
    model='gpt-4o-2024-08-06',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_a288987b44',
    usage=CompletionUsage(
        completion_tokens=26,
        prompt_tokens=69,
        total_tokens=95,
        prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0},
        completion_tokens_details={
            'reasoning_tokens': 0,
            'audio_tokens': 0,
            'accepted_prediction_tokens': 0,
            'rejected_prediction_tokens': 0
        }
    )
)

In [67]:
second_response.choices[0].message.content

'100 USD는 약 130,000 KRW입니다. 환율은 변동될 수 있으니 확인이 필요합니다.'

---

### 전체 코드 실행

In [68]:
# --- 준비 과정 (날씨 요정과 동일) ---
# client = openai.OpenAI(api_key="여러분의_API_키")
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 새로운 도구 제작: '환율 계산기 로봇' ---
# 3가지 정보(금액, 시작 통화, 목표 통화)를 받아 계산하는 점이 다릅니다.
def get_exchange_rate(amount, from_currency, to_currency):
    """지정된 금액을 한 통화에서 다른 통화로 변환하는 가짜 함수입니다."""
    print(f"🛠️ [도구 실행] 'get_exchange_rate' 로봇이 {amount} {from_currency}를 {to_currency}로 변환합니다.")
    exchange_rates = {"USD_to_KRW": 1300, "EUR_to_KRW": 1400}
    conversion_key = f"{from_currency}_to_{to_currency}"
    rate = exchange_rates.get(conversion_key)
    if rate:
        converted_amount = amount * rate
        return json.dumps({"amount": converted_amount, "currency": to_currency})
    else:
        return json.dumps({"error": "지원하지 않는 통화 변환입니다."})

# --- 에이전트 실행 (핵심 구조는 날씨 요정과 동일) ---
def run_exchange_agent(user_question):
    print(f"😎 사용자 질문: {user_question}\n")
    print("🧠 [1차 호출] AI, 이 질문을 보고 필요한 정보를 뽑아서 도구를 쓸지 결정해줘!")
    
    # 여기서는 '환율 계산기'의 '업그레이드된 신청서'를 제출합니다.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": user_question}],
        tools=[{
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "한 통화의 금액을 다른 통화로 환전해줍니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "amount": {"type": "number", "description": "환전할 금액"},
                        "from_currency": {"type": "string", "description": "환전의 기준이 되는 통화 (예: USD, EUR)"},
                        "to_currency": {"type": "string", "description": "환전 후의 목표 통화 (예: KRW)"}
                    },
                    "required": ["amount", "from_currency", "to_currency"],
                },
            }
        }],
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        # AI가 하나의 문장에서 3개의 정보를 정확히 추출했는지 확인해보세요!
        print("✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요!")
        tool_call = tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        print(f"   - AI가 추출한 정보: {function_args}\n")

        # 이하 도구 실행 및 2차 호출 과정은 날씨 요정과 완전히 동일합니다.
        messages = [{"role": "user", "content": user_question}, response_message]
        function_response = get_exchange_rate(
            amount=function_args.get("amount"),
            from_currency=function_args.get("from_currency"),
            to_currency=function_args.get("to_currency"),
        )
        print(f"📄 [도구 결과] 계산 결과: {function_response}\n")
        
        print("🧠 [2차 호출] AI, 계산 결과를 바탕으로 최종 답변을 만들어줘!")
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": "get_exchange_rate",
            "content": function_response,
        })
        second_response = client.chat.completions.create(model="gpt-4o", messages=messages)
        return second_response.choices[0].message.content
    else:
        return response_message.content

# --- 최종 실행 및 성공 확인 ---
final_result = run_exchange_agent("150달러는 한국 돈으로 얼마야?")
print("\n🎉 [최종 답변] 환율 계산기 로봇의 답변입니다!")
print(final_result)

😎 사용자 질문: 150달러는 한국 돈으로 얼마야?

🧠 [1차 호출] AI, 이 질문을 보고 필요한 정보를 뽑아서 도구를 쓸지 결정해줘!
✅ AI 응답: '환율 계산기 로봇'을 사용해야겠어요!
   - AI가 추출한 정보: {'amount': 150, 'from_currency': 'USD', 'to_currency': 'KRW'}

🛠️ [도구 실행] 'get_exchange_rate' 로봇이 150 USD를 KRW로 변환합니다.
📄 [도구 결과] 계산 결과: {"amount": 195000, "currency": "KRW"}

🧠 [2차 호출] AI, 계산 결과를 바탕으로 최종 답변을 만들어줘!

🎉 [최종 답변] 환율 계산기 로봇의 답변입니다!
150달러는 약 195,000원입니다. 환율은 변동될 수 있으니, 실제 거래 시 최신 환율을 확인하시기 바랍니다.
